In [2]:
!nvidia-smi

Thu Feb 24 19:43:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -qU transformers
!pip install -qU hazm

     |████████████████████████████████| 3.5 MB 4.3 MB/s 
     |████████████████████████████████| 6.8 MB 37.5 MB/s 
     |████████████████████████████████| 596 kB 49.3 MB/s 
     |████████████████████████████████| 895 kB 50.3 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 316 kB 4.1 MB/s 
     |████████████████████████████████| 233 kB 45.5 MB/s 
     |████████████████████████████████| 1.4 MB 41.1 MB/s 


In [5]:
import json
import os
import glob

import numpy as np
import pandas as pd

from tqdm import tqdm

In [ ]:
class Data_load:
  def __init__(self,max_length, step):
    self.vocab = []
    self.max_length = max_length
    self.step = step
    # self.load_dictionary()

    
  def create_sequence_data(self,paths):
    poems =  self.read_data(paths)
    all_poem_seq = []
    for poem_seq in poems:
        word2id_sequence = []
        words_list = poem_seq.split(" ")
        # print(words_list)
        for word in words_list:

          try:
              word2id_sequence.append(self.dict_word2id[word])
          except KeyError:
              word2id_sequence.append(self.dict_word2id["UNK"])
        all_poem_seq.append(word2id_sequence)
    X = []
    Y = []
    for poem_seq in all_poem_seq:
      for i in range(self.max_length,len(poem_seq),self.step):
         X.append(poem_seq[i - self.max_length: i])
         Y.append(poem_seq[i - self.max_length+1: i+1])
    return X,Y
            
  def read_data(self,paths):
    poems_sequence = []
    poems = pd.read_csv(paths[0])
    i = 1
    while True:
            poem_i = poems[poems['poem_id'] == i]
            # print(poem_i)
            index_i = poems.index[poems['poem_id'] == i]
            current_poem = ""
            for p in index_i:
                v_position = poem_i.loc[p,"v_position"]
                verse =  poem_i.loc[p,"poem_text"]
                # print(verse)
                current_poem += verse 
                if v_position == 0:
                    current_poem += " \t"
                if v_position == 1:
                    current_poem += " \n"
            if len(current_poem)>0:
              poems_sequence.append(current_poem)
            i += 1
            if i>599 :
              break
    for path in paths[1:]:
        
        poems = pd.read_csv(path)
        # print(poems['poem_id'])
        i = 1
        while True:
            poem_i = poems[poems['poem_id'] == i]
          
            index_i = poems.index[poems['poem_id'] == i]
            current_poem = ""
            for p in index_i:
                v_position = poem_i.loc[p,"v_position"]
                verse =  poem_i.loc[p,"poem_text"]
                current_poem += verse 
                if v_position == 0:
                    current_poem += " \t"
                if v_position == 1:
                    current_poem += " \n"
            poems_sequence.append(current_poem)
            i += 1
            if len(poem_i)<1 :
              break
    return poems_sequence
import os
paths = os.listdir("/content/drive/MyDrive/NLU_Project/Data/poems/")
# print(paths)
paths_arr = ["/content/drive/MyDrive/NLU_Project/Data/poems/" + f for f in paths]
d_load = Data_load(60,2)
poems= d_load.read_data(["/content/drive/MyDrive/NLU_Project/Data/train.csv"]+paths_arr)
X , Y  = [], []
# persian_poems = open("/content/drive/MyDrive/persian_poems_test.txt",'w')
for poem in poems:
  for vers in poem.split("\n"):
    try:
      x,y = vers.split("\t")
      X.append(x.strip())
      Y.append(y.strip())
    except ValueError:
      continue

In [ ]:
X = X[-100000:]
Y = Y[-100000:]

In [ ]:
b = []
for i in range(len(X)):
  a = []
  a.append(X[i])
  a.append(Y[i])
  b.append(a)

In [ ]:
b[0]

['دیدم رخت از غم سر موئیم نماند', 'جز بندگی روی تو روئیم نماند']

In [ ]:
sentence = []
for i in range(len(b)):
  temp = ' '.join(b[i])
  sentence.append(temp)

In [ ]:
sentence[:5]

['دیدم رخت از غم سر موئیم نماند جز بندگی روی تو روئیم نماند',
 'با دل گفتم که آرزوئی در خواه دل گفت که هیچ آرزوئیم نماند',
 'دی می\u200cرفتی بر تو تو نظر می\u200cکردند آنانکه به مذهب تناسخ فردند',
 'سوگند به اعتقاد خود می\u200cخوردند کاین یوسف ثانیست که باز آوردند',
 'دیوانه میان خلق پیدا باشد زیرا که سوار اسب سودا باشد']

In [ ]:
from transformers import BertTokenizer, BertForNextSentencePrediction,BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')
model = BertForMaskedLM.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')
# tokenizer.add_special_tokens({
#     "bos_token": '</s>',
#     "eos_token": '</s>', 
#     "pad_token": '<pad>',
#     "unk_token": '<unk>'
# })
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
inputs = tokenizer(sentence, return_tensors='pt', max_length=40, truncation=True, padding='max_length')
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
rand = torch.rand(inputs.input_ids.shape)

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [1]:

# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 2) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

NameError: ignored

In [ ]:

selection = []

for i in range(mask_arr.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
for i in range(mask_arr.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()

In [ ]:
from transformers import AdamW
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/12500 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 1: 100%|██████████| 12500/12500 [1:09:33<00:00,  2.99it/s, loss=0.442]


In [ ]:
model.save_pretrained('/content/drive/MyDrive/my_model_MSK')

In [6]:
poems = pd.read_csv("/content/drive/MyDrive/NLU_Project/Data/test.csv")

i = 3
poems_sequence =[]
while True:
            poem_i = poems[poems['poem_id'] == i]
            # print(poem_i)
            index_i = poems.index[poems['poem_id'] == i]
            current_poem = ""
            for p in index_i:
                v_position = poem_i.loc[p,"v_position"]
                verse =  poem_i.loc[p,"poem_text"]
                # print(verse)
                current_poem += verse 
                if v_position == 0:
                    current_poem += " \t"
                if v_position == 1:
                    current_poem += " \n"
            if len(current_poem)>0:
              poems_sequence.append(current_poem)
            i += 1
            if i>596 :
              break

# X , Y  = [], []
# for poem in poems_sequence:
#   print(poem)
#   if len(poem)==0:
#     break
#   for vers in poem.split("\n"):
#     s = "[CLS] "
#     try:
#       x,y = vers.split("\t")
#       l = len(x.split(" "))
#       s += x+" [SEP]"
#       for i in range(l):
#         s+= " [MASK] "
#       s += "[SEP]"
#       X.append(s)
#       Y.append(y.strip())
#     except ValueError:
#       continue

X , Y  = [], []
for poem in poems_sequence:
  # print(poem)
  if len(poem)==0:
    break

  for  vers in poem.split("\n"):
    s = "[CLS] "
    try:
      x,y =vers.split("\t")
      l = len(x.split(" "))
      s += x+" [SEP]"+" [MASK]"
      for i in range(l):
        s+= " [MASK] "
      s += "[SEP]"
      X.append(s)
      Y.append(y.strip())
    except ValueError:
      continue
    except IndexError:
      continue

In [10]:
X[1]

'[CLS] گفتم سخن تو، گفت حافظ گفتا  [SEP] [MASK] [MASK]  [MASK]  [MASK]  [MASK]  [MASK]  [MASK]  [MASK] [SEP]'

In [7]:
from transformers import BertTokenizer, BertForNextSentencePrediction,BertForMaskedLM
import numpy as np
import torch
import torch.nn as nn
from abc import abstractmethod
import math
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim
from torch.distributions.categorical import Categorical

In [8]:
MASK = "[MASK]"
MASK_ATOM = "[MASK]"
tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
def preprocess(tokens, tokenizer, device):
    """ Preprocess the sentence by tokenizing and converting to tensor """
    tok_ids = tokenizer.convert_tokens_to_ids(tokens)
    tok_tensor = torch.tensor([tok_ids])
    tok_tensor = tok_tensor.to(device)
    return tok_tensor


def get_mask_ids(masking):
    if masking:
      mask_ids = [int(d) for d in masking.split(',')]
    else:
      mask_ids = []     
    return mask_ids

  
def get_seed_sent(seed_sentence, tokenizer, masking=None, n_append_mask=0):
    """ Get initial sentence to decode from, possible with masks """

    # Get initial mask
    mask_ids = get_mask_ids(masking)

    # Tokenize, respecting [MASK]
    seed_sentence = seed_sentence.replace(MASK, MASK_ATOM)
    toks = tokenizer.tokenize(seed_sentence)
    for i, tok in enumerate(toks):
        if tok == MASK_ATOM:
            mask_ids.append(i)

    # Mask the input
    for mask_id in mask_ids:
        toks[mask_id] = MASK

    # Append MASKs
    for _ in range(n_append_mask):
        mask_ids.append(len(toks))
        toks.append(MASK)
    mask_ids = sorted(list(set(mask_ids)))

    seg = [0] * len(toks)
    seg_tensor = torch.tensor([seg])
    return toks, seg_tensor, mask_ids

  
def load_model(version):
    """ Load model """
    model = BertForMaskedLM.from_pretrained(version)
    model.eval()
    return model


def predict(model, tokenizer, tok_tensor, seg_tensor, how_select="argmax"):
    """ Get model predictions and convert back to tokens """
    preds = model(tok_tensor, seg_tensor)
    # print(preds.logits.size())
    # print(torch.argmax(preds.logits.squeeze(0),dim=1))
    
    if how_select == "sample":
        dist = Categorical(logits=F.log_softmax(preds[0], dim=-1))
        pred_idxs = dist.sample().tolist()
    elif how_select == "topk":
        kth_vals, kth_idx = F.log_softmax(preds[0], dim=-1).topk(30, dim=-1)
        dist = Categorical(logits=kth_vals)
        pred_idxs = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1).tolist()
    elif how_select == "argmax":
        pred_idxs = torch.argmax(preds.logits.squeeze(0),dim=1).tolist()
    else:
        raise NotImplementedError("Prediction procedure %s not found!" % how_select)

    pred_toks = tokenizer.convert_ids_to_tokens(pred_idxs[0])
    # print(pred_idxs)
    return pred_toks
  

def masked_decoding(toks, seg_tensor, masks, model, tokenizer, device, selection_strategy):
    """ Decode from model by replacing masks """
    for step_n, mask_id in enumerate(masks):
        tok_tensor = preprocess(toks, tokenizer, device)
        pred_toks = predict(model, tokenizer, tok_tensor, seg_tensor, selection_strategy)
        # print(pred_toks)
        toks[mask_id] = pred_toks[mask_id]
    return toks

  
def detokenize(pred_toks):
    """ Return the detokenized lyric prediction """
    new_sent = []
    for i, tok in enumerate(pred_toks):
        if tok.startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
@torch.no_grad()
def main():
  score1 = 0
  score2 = 0
  
  score3 = 0
  masked_model = load_model('/content/drive/MyDrive/my_model_MSK').to(device)
  for s,y in zip(X[:500],Y[:500]):
    g_poems = s
    # for j in range(10):
    toks, seg_tensor, mask_ids = get_seed_sent(g_poems,
                                               tokenizer,
                                               masking=None,
                                               n_append_mask=0)
    seg_tensor = seg_tensor.to(device)
    pred_toks = masked_decoding(toks, seg_tensor, mask_ids, masked_model, tokenizer, device, "sample")
    pred_lyric = detokenize(pred_toks)
        # if j !=9:
          # g_poems = " ".join(pred_lyric)+" [MASK]"
    print(" ".join(pred_lyric).replace("[UNK]","\t"))
    print(y)
    print("===============================================================")
    s1 = sentence_bleu([f.split(" ")for f in Y[:500]]," ".join(pred_lyric).replace("[UNK]","\t")[pred_lyric.index("[SEP]")+5:].split(" "),weights= (1,0,0,0)) 
    s2 = sentence_bleu([f.split(" ")for f in Y[:500]]," ".join(pred_lyric).replace("[UNK]","\t")[pred_lyric.index("[SEP]")+5:].split(" "),weights= (0,1,0,0)) 
    s3 = sentence_bleu([f.split(" ")for f in Y[:500]]," ".join(pred_lyric).replace("[UNK]","\t")[pred_lyric.index("[SEP]")+5:].split(" "),weights= (0,0,1,0)) 
    score1 += s1
    score2 += s2
    score3 += s3
    print(s1,s2,s3)
  print(score1/500,score2/500,score3/500)
 
if __name__ == '__main__':
    main()

Downloading:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/434 [00:00<?, ?B/s]

[CLS] گفتم که لبت ، گفت لبم اب حیات [SEP] گفت لب شیر ! کسزهست هر زمان [SEP]
گفتم دهنت، گفت زهی حب نبات
0.4 2.2250738585072626e-308 2.2250738585072626e-308


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

[CLS] گفتم سخن تو ، گفت حافظ گفتا [SEP] کرد پشیمانی نبود ز گفتار نگنجد گفتار [SEP]
شادی همه لطیفه گویان صلوات
0.4666666666666667 2.2250738585072626e-308 2.2250738585072626e-308
[CLS] ای کاش که بخت سازگاری کردی [SEP] ناخلف گذشتیم زین بلندی و ازادگی اندر کنار [SEP]
با جور زمانه یار یاری کردی
0.4666666666666667 2.2250738585072626e-308 2.2250738585072626e-308
[CLS] از دست جوانیام چو بربود عنان [SEP] کشم به دست عقل بین مرا ناظرم [SEP]
پیری چو رکاب پایداری کردی
0.42857142857142855 0.07692307692307693 2.2250738585072626e-308
[CLS] دادگرا تو را فلک جرعه کش پیاله باد [SEP] خشم از دوست من ترا هوای بازم اقبال ازاد باد [SEP]
دشمن دل سیاه تو غرقه به خون چو لاله باد
0.631578947368421 0.05555555555555554 2.2250738585072626e-308
[CLS] ذروهی کاخ رتبتت راست ز فرط ارتفاع [SEP] خدمت راست ز خداوندی وپهم خاموشیت [SEP]
راهروان وهم را راه هزار ساله باد
0.14285714285714285 2.2250738585072626e-308 2.2250738585072626e-308
[CLS] ای مه برج منزلت چشم و چراغ عالمی [SEP] از چه کار می باک که خورشید را شجری باشد [SEP]
